In [ ]:
!pip install kagglehub torch torchvision torchaudio transformers datasets scikit-learn tqdm


In [ ]:
import kagglehub
import os
import pandas as pd

# Veri setini indir
path = kagglehub.dataset_download("bittlingmayer/amazonreviews")
print("Path to dataset files:", path)

# Dosyaları inceleyelim
os.listdir(path)


100%|██████████| 493M/493M [00:24<00:00, 20.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/bittlingmayer/amazonreviews/versions/7


['test.ft.txt.bz2', 'train.ft.txt.bz2']

In [ ]:
import bz2

def read_fasttext(path, limit=None):
    texts, labels = [], []
    with bz2.open(path, "rt", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if limit and i >= limit:
                break
            label, text = line.split(" ", 1)
            labels.append(1 if "__label__2" in label else 0)
            texts.append(text.strip())
    return pd.DataFrame({"text": texts, "label": labels})


train_df = read_fasttext(os.path.join(path, "train.ft.txt.bz2"), limit=20000)
test_df = read_fasttext(os.path.join(path, "test.ft.txt.bz2"), limit=5000)

print(train_df.head())
print(train_df['label'].value_counts())


                                                text  label
0  Stuning even for the non-gamer: This sound tra...      1
1  The best soundtrack ever to anything.: I'm rea...      1
2  Amazing!: This soundtrack is my favorite music...      1
3  Excellent Soundtrack: I truly like this soundt...      1
4  Remember, Pull Your Jaw Off The Floor After He...      1
label
1    10257
0     9743
Name: count, dtype: int64


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(
    train_df["text"].tolist(), truncation=True, padding=True, max_length=128
)
test_encodings = tokenizer(
    test_df["text"].tolist(), truncation=True, padding=True, max_length=128
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import torch

class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = ReviewDataset(train_encodings, train_df["label"].tolist())
test_dataset = ReviewDataset(test_encodings, test_df["label"].tolist())


In [ ]:
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification
from torch.optim import AdamW


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tqdm import tqdm

model.train()
for epoch in range(2):
    total_loss = 0
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())
    print(f"Epoch {epoch+1} Avg Loss: {total_loss/len(train_loader):.4f}")


Epoch 1: 100%|██████████| 1250/1250 [07:00<00:00,  2.97it/s, loss=0.267]


Epoch 1 Avg Loss: 0.2082


Epoch 2: 100%|██████████| 1250/1250 [07:02<00:00,  2.96it/s, loss=0.0376]

Epoch 2 Avg Loss: 0.1019


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

model.eval()
preds, true_labels = [], []

with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print("✅ Test Accuracy:", accuracy_score(true_labels, preds))
print(classification_report(true_labels, preds, target_names=["Negative", "Positive"]))


100%|██████████| 157/157 [00:31<00:00,  4.96it/s]

✅ Test Accuracy: 0.9336
              precision    recall  f1-score   support

    Negative       0.95      0.91      0.93      2435
    Positive       0.92      0.96      0.94      2565

    accuracy                           0.93      5000
   macro avg       0.94      0.93      0.93      5000
weighted avg       0.93      0.93      0.93      5000



In [ ]:
def predict_sentiment(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    tokens = {k: v.to(device) for k, v in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
        pred = torch.argmax(outputs.logits, dim=-1).item()
    return "Positive 😀" if pred == 1 else "Negative 😞"

print(predict_sentiment("This product is amazing!"))
print(predict_sentiment("Terrible quality and bad packaging."))


Positive 😀
Negative 😞


In [ ]:
model.save_pretrained("amazon_sentiment_bert")
tokenizer.save_pretrained("amazon_sentiment_bert")


('amazon_sentiment_bert/tokenizer_config.json',
 'amazon_sentiment_bert/special_tokens_map.json',
 'amazon_sentiment_bert/vocab.txt',
 'amazon_sentiment_bert/added_tokens.json')

In [ ]:
!pip install huggingface_hub transformers torch


In [ ]:
from huggingface_hub import login
login()


In [ ]:
from huggingface_hub import create_repo, upload_folder

username = "birolshn"
repo_name = "amazon-sentiment-bert"
repo_id = f"{username}/{repo_name}"


create_repo(repo_id, exist_ok=True)


upload_folder(
    folder_path="amazon_sentiment_bert",
    repo_id=repo_id,
    repo_type="model"
)

print(f"✅ Model successfully uploaded: https://huggingface.co/{repo_id}")


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...nt_bert/model.safetensors:   0%|          | 14.2kB /  438MB            

✅ Model successfully uploaded: https://huggingface.co/birolshn/amazon-sentiment-bert


In [ ]:
!curl -X POST https://birolshn-amazon-sentiment-api.hf.space/api/predict \
     -H "Content-Type: application/json" \
     -d '{"data": ["This product is great!"]}'



{"data":[[{"label":"negative","score":0.9941583871841431}]]}
